In [3]:
# requirements 
from __future__ import unicode_literals, print_function, division
from io import open 
import unicodedata
import re 
import random 

import torch 
import torch.nn as nn 
from torch import optim 
import torch.nn.functional as F 

import numpy as np 
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Create a unique index per word to use as inputs and targets of the networks later.

In [14]:
SOS_token = 0 
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2 # This is the word count

    def addSentence(self, sentence): 
        for word in sentence.split(" "):
            self.addWord(word)

    def addWord(self, word): 
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1 # Update the word count
        else :
            self.word2count[word] += 1

In [4]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return "".join(
        c for c in unicodedata.normalize("NFD", s)
        if unicodedata.category(c) != "Mn"
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", " \1", s)
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s.strip()

In [40]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s-cleaned.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs  # Add this line


In [44]:
# Only include sentences with max 10 words for quick testing.
# Increase for more complexity
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p): 
    return len(p[0].split(" ")) < MAX_LENGTH and \
        len(p[1].split(" ")) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)
    
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [45]:
# Remove CC lines
# Initialize an empty list to store the processed lines
processed_lines = []

# Open the file and read line by line
with open("./data/en-it.txt", "r", encoding="utf-8") as file:
    for line in file:
        # Find the index of "CC-BY"
        index = line.find("CC-BY")
        
        # If "CC-BY" is found in the line
        if index != -1:
            # Remove "CC-BY" and everything after it and add the processed line to the list
            processed_lines.append(line[:index].strip())
        else:
            # If "CC-BY" is not found, add the original line to the list
            processed_lines.append(line.strip())

# Join the processed lines into a single string
processed_text = "\n".join(processed_lines)

# Write the processed text to a new file
with open("./data/en-it-cleaned.txt", "w", encoding="utf-8") as file:
    file.write(processed_text)

In [46]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))

    pairs = filterPairs(pairs)

    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")

    for pair in pairs: 
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)

    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('en', 'it', reverse=True)
print(random.choice(pairs))

Reading lines...
Read 374932 sentence pairs
Trimmed to 34220 sentence pairs
Counting words...
Counted words:
it 5670
en 3340
['sono preoccupato', 'i m worried']


In [ ]:
class PositionalEncoding(nn.Module):